In [1]:
from typing import List
from math import pi
import rospy
from copy import deepcopy
from sensor_msgs.msg import Image

from moveit_commander import PlanningSceneInterface

from cv_bridge import CvBridge

from geometry_msgs.msg import (
    Pose,
    PoseStamped,
    Point,
    Quaternion,
)
from commander.msg import Goal
from commander.srv import (
    ExecuteTrajectory,
    PlanGoal,
    PlanGoalRequest,
    PlanSequence,
    PlanSequenceRequest,
    PickPlace,
    GetTcpPose,
    VisualizePoses,
    SetEe,
)

from commander.utils import poses_from_yaml, load_scene
from commander.transform_utils import orient_poses, create_rotation_matrix, apply_transformation


import cv2
import numpy as np

plan_goal_srv = rospy.ServiceProxy('commander/plan_goal', PlanGoal)
plan_sequence_srv = rospy.ServiceProxy('commander/plan_sequence', PlanSequence)
execute_trajectory_srv = rospy.ServiceProxy('commander/execute_trajectory', ExecuteTrajectory)
get_tcp_pose_srv = rospy.ServiceProxy('commander/get_tcp_pose', GetTcpPose)
set_ee_srv = rospy.ServiceProxy('commander/set_ee', SetEe)
pick_place_srv = rospy.ServiceProxy('commander/pick_place', PickPlace)


def display_poses(poses: List[Pose], frame_id: str = 'base_link') -> None:
    rospy.wait_for_service('/visualize_poses', timeout=10)
    visualize_poses = rospy.ServiceProxy('/visualize_poses', VisualizePoses)
    visualize_poses(frame_id, poses)


rospy.init_node('robot_program')
scene = PlanningSceneInterface()
load_scene()


In [5]:
cam_home = [-3.1416, -1.5708, -1.5708, -3.1416, -1.5708, -3.1416]


In [6]:
plan_goal_srv(Goal(joint_values=cam_home, vel_scale=0.2, acc_scale=0.2, planner='ptp'))

success: True
configuration_change: False

In [17]:
img = cv2.imread("/dev_ws/src/images/qr4.png")
# aruco macker detection
arucoDict = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_4X4_50)

arucoParams = cv2.aruco.DetectorParameters()

detector = cv2.aruco.ArucoDetector(arucoDict, arucoParams)

corners, ids, rejected = detector.detectMarkers(img)

print(ids)

# if the ID of marker is 0, then actuate the robot

if ids is not None and ids[0] == 0:
    success = set_ee_srv('rgb_camera_tcp')
    target0 = Pose(
        position=Point(0.5, -0.4, 0.4),
        orientation=Quaternion(0.7071067811865476, -0.7071067811865475, 0.0, 0.0),
    )
    target1 = Pose(
        position=Point(0.5, 0.4, 0.4),
        orientation=Quaternion(0.7071067811865476, -0.7071067811865475, 0.0, 0.0),
    )
    display_poses([target0, target1])

    success = plan_goal_srv(
        Goal(
            pose=target0,
            vel_scale=0.2,
            acc_scale=0.2,
            planner='ptp',
        )
    ).success


None


In [ ]:
success = execute_trajectory_srv()